<a href="https://colab.research.google.com/github/ericdelmelle/SpatialOptimization/blob/main/Knapsack_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example of use of the Pulp libray for a simple knapsack problem


In [2]:
#Install pulp
!pip install pulp

# Import libraries
from pulp import *
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 27.7 MB/s eta 0:00:00


Let set up our problem with the decision variables (items),
with their **value** "v" and **weight** "w". We then set up the **capacity** of our bagpack.

In [4]:
# A list of tuples of items (value, weight)
#
items = [(20,5), (30,6), (10,7), (90,32), (10,2), (40,5), (100,7), (60,9), (70,12), (50,11), (30,1), (20,2)]

# number of items
itemCount = len(items)

# Knapsack max weight capacity
binCapacity = 32


Here we set up our decision variables (the X), the lower and upper bounds, couple with the integer constrains (cat=Integer)  tells us the decision variable will be binary.

In [5]:
# Decision variables (array), x[i] gets 1 when item i is included in the solution
x = pulp.LpVariable.dicts('item', range(itemCount),
                            lowBound = 0,
                            upBound = 1,
                            cat = 'Integer')

Here we set up our problem (it is of maximization type, and it is a linear programming problem).

We add the objective function (sum of the values), and the constraint (sum of the weights).

In [6]:
# Initialize the problem and specify the type
problem = LpProblem("Knapsack Problem", LpMaximize)

# Add the objective function
problem += lpSum([ x[i] * (items[i])[0] for i in range(itemCount) ]), "Objective: Maximize profit"

# Capacity constraint: the sum of the weights must be less than the capacity
problem += lpSum([ x[i] * (items[i])[1] for i in range(itemCount) ]) <= binCapacity, "Constraint: Max capacity"

Here we write the problem as an lp file and we then solve it, using the "**solve**" function. We also keep track of the time it takes.

In [7]:
#print problem.constraints

# Write the model to disk, not necessary
problem.writeLP("Knapsack.lp")

# Solve the optimization problem
start_time = time.time()
problem.solve()
print("Solved in %s seconds." % (time.time() - start_time))

Solved in 0.015312910079956055 seconds.


Finally, we report on the optimal solution, and whether each variable was chosen or not

In [8]:
# Was the problem solved to optimality?
print("Status:", LpStatus[problem.status])

# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total profit = ", value(problem.objective))

Status: Optimal
item_0 = 0.0
item_1 = 1.0
item_10 = 1.0
item_11 = 1.0
item_2 = 0.0
item_3 = 0.0
item_4 = 1.0
item_5 = 1.0
item_6 = 1.0
item_7 = 1.0
item_8 = 0.0
item_9 = 0.0
Total profit =  290.0


Here, somore more info about the solution

In [11]:
used_cap = 0.0
print("Used items:")
for i in range(itemCount):
    if x[i].value() == 1:
        print(i, items[i])
        used_cap += items[i][1]
print("Profit: %d - Used capacity: %d (/ %d)" % (value(problem.objective), used_cap, binCapacity))



Used items:
1 (30, 6)
4 (10, 2)
5 (40, 5)
6 (100, 7)
7 (60, 9)
10 (30, 1)
11 (20, 2)
Profit: 290 - Used capacity: 32 (/ 32)
